In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import re
from sklearn.utils import shuffle
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import model_from_json,load_model,Model 



Using TensorFlow backend.


In [2]:
lines=pd.read_table('eng-mar-dataset.txt',names=['eng', 'mar','waste'])
del lines['waste']
print(lines.shape)

lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())


lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation)

lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


(38696, 2)


In [3]:
lines.sample(10)

,eng,mar
32532,i dont like the taste of tomatoes,START_ मला टोमॅटोची चव आवडत नाही _END
18463,greece has many islands,START_ ग्रीसमध्ये अनेक बेटं आहेत _END
5889,its monkey meat,START_ ते माकडाचं मांस आहे _END
22508,my coffee mug disappeared,START_ माझा कॉफी मग गायब झाला _END
29837,why do you want to leave today,START_ तुला आज का निघायचं आहे _END
38676,our website is offline for scheduled maintenan...,START_ आमचे संकेतस्थळ नियोजित देखरेखीकरिता ऑफल...
7602,she is aggressive,START_ ती आक्रमक आहे _END
31347,tom said that he liked the movie,START_ टॉम म्हणाला की त्याला चित्रपट आवडला _END
30164,i have watched all of her films,START_ मी त्यांचे सगळे चित्रपट पाहिले आहेत _END
17679,tom already knows this,START_ टॉमला हे आधीच माहीत आहे _END


In [4]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)


all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)
            
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(5585, 13386)

In [5]:
num_decoder_tokens += 1
num_decoder_tokens

13387

In [6]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
lines = shuffle(lines)

In [7]:
json_file = open('model_2.json','r')
loaded_model_json = json_file.read()
json_file.close()
model_loaded = model_from_json(loaded_model_json)

model_loaded.load_weights('nmt_weights.h5')

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
latent_dim = 50

encoder_inputs_inf = model_loaded.input[0] 
encoder_outputs_inf, inf_state_h, inf_state_c = model_loaded.layers[4].output
encoder_inf_states = [inf_state_h,inf_state_c]
encoder_model = Model(encoder_inputs_inf,encoder_inf_states)

In [10]:
decoder_state_h_input = Input(shape=(latent_dim,)) 
decoder_state_c_input = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_h_input,decoder_state_c_input]


decoder_input_inf = model_loaded.input[1] 

decoder_emb_inf = model_loaded.layers[3](decoder_input_inf)
decoder_lstm_inf = model_loaded.layers[5]
decoder_output_inf, decoder_state_h_inf, decoder_state_c_inf = decoder_lstm_inf(decoder_emb_inf, initial_state =decoder_state_input)
decoder_state_inf = [decoder_state_h_inf,decoder_state_c_inf]

dense_inf = model_loaded.layers[6]
decoder_output_final = dense_inf(decoder_output_inf)

decoder_model = Model([decoder_input_inf]+decoder_state_input,[decoder_output_final]+decoder_state_inf)

In [11]:
def decode_sequence(input_seq):
    
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1))
    
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        
        states_value = [h, c]

    return decoded_sentence

In [12]:
import speech_recognition as sr
from gtts import gTTS 
import os

In [13]:
def prediction(a):
    
    a=a.lower()
    l=np.zeros((1,max_length_src),dtype='float32')
    
    a=a.rstrip().split()
    
    for i in range(len(a)):
        l[0][i]=input_token_index[a[i]]
    return (decode_sequence(l)[:-4])


In [ ]:
r=sr.Recognizer()

with sr.Microphone() as source:
    print("speak anything: ")
    audio=r.listen(source)
    
    try:
        text = r.recognize_google(audio)
        print(text)
        print(prediction(text))
        tts=gTTS(text=prediction(text),lang='hi',slow=False)
        tts.save('conv.mp3')
        os.system('conv.mp3')
        
    except:
        tts=gTTS(text="could not recognize the input",lang='en',slow=False)
        tts.save('conv1.mp3')
        os.system('conv1.mp3')

speak anything: 
